<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/Sector_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2024-02-03 23:09:58


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-10*365)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
# fetch nse indices scrips

import pandas as pd
import numpy as np

df_nse_sectors = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/NSE_Sectors_Indices.csv')

In [40]:
sectors = ['FMCG'] #df_nse_sectors['Sector'].unique()
df_sectors = pd.DataFrame()
for sector in sectors:
  stocks = df_nse_sectors[df_nse_sectors['Sector'] == sector]['Symbol'].values
  df_sector = pd.DataFrame()
  tweight = 0
  for stock in stocks:
    stock_df = yfin.Ticker(stock+'.NS').history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    col = ['Close']
    stock_df = stock_df[col]
    stock_df['Split'] = 1
    weight = df_nse_sectors[df_nse_sectors['Symbol'] == stock]['Weight'].values[0]
    tweight = sum(df_nse_sectors[df_nse_sectors['Sector'] == sector]['Weight'].values)
    if stock == 'NESTLEIND':
      stock_df.loc[stock_df.index < '2024-01-01', 'Split'] = 10
    stock_df[stock] = (stock_df[col]*weight)/(tweight*stock_df['Split'].values[0])
    col = [stock]
    stock_df = stock_df[col]
    df_sector = pd.concat([df_sector, stock_df], axis=1)

  df_sector[sector] = df_sector[stocks].sum(axis=1)
  #col = [sector]
  #df_sector = df_sector[col]
  df_sectors = pd.concat([df_sectors, df_sector], axis=1)
df_sectors

,ITC,HINDUNILVR,NESTLEIND,TATACONSUM,VBL,BRITANNIA,FMCG
Date,,,,,,,
2019-02-04 00:00:00+05:30,92.084237,429.689372,111.078433,16.407056,8.639486,222.655999,880.554582
2019-02-05 00:00:00+05:30,91.103899,434.182671,111.788232,15.265908,8.586563,215.910179,876.837452
2019-02-06 00:00:00+05:30,92.283618,436.208879,111.884042,15.471151,9.015890,216.888508,881.752089
2019-02-07 00:00:00+05:30,93.147636,438.378096,112.830770,15.688708,8.832818,219.117773,887.995801
2019-02-08 00:00:00+05:30,91.552531,433.193470,108.275415,15.245385,8.798796,212.020874,869.086470
...,...,...,...,...,...,...,...
2024-01-29 00:00:00+05:30,180.354033,633.135577,26.108516,97.791872,95.264611,384.158715,1416.813323
2024-01-30 00:00:00+05:30,175.642694,636.826536,26.040594,96.236353,93.887655,380.782218,1409.416051
2024-01-31 00:00:00+05:30,177.046072,642.758012,26.186364,95.816404,96.078773,388.757941,1426.643565


In [41]:
df_sectors[df_sectors.index < '2024-01-01']

,ITC,HINDUNILVR,NESTLEIND,TATACONSUM,VBL,BRITANNIA,FMCG
Date,,,,,,,
2019-02-04 00:00:00+05:30,92.084237,429.689372,111.078433,16.407056,8.639486,222.655999,880.554582
2019-02-05 00:00:00+05:30,91.103899,434.182671,111.788232,15.265908,8.586563,215.910179,876.837452
2019-02-06 00:00:00+05:30,92.283618,436.208879,111.884042,15.471151,9.015890,216.888508,881.752089
2019-02-07 00:00:00+05:30,93.147636,438.378096,112.830770,15.688708,8.832818,219.117773,887.995801
2019-02-08 00:00:00+05:30,91.552531,433.193470,108.275415,15.245385,8.798796,212.020874,869.086470
...,...,...,...,...,...,...,...
2023-12-22 00:00:00+05:30,182.519253,667.118359,265.086766,85.086337,92.942177,385.968507,1678.721400
2023-12-26 00:00:00+05:30,183.020459,669.488319,267.109799,86.333321,93.801369,391.566108,1691.319375
2023-12-27 00:00:00+05:30,183.281084,675.588128,268.764979,87.995969,92.825873,390.133962,1698.589995


In [36]:
df_sectors[df_sectors.index > '22-12-2023']

,ITC,HINDUNILVR,NESTLEIND,TATACONSUM,VBL,BRITANNIA,FMCG
Date,,,,,,,
2023-12-26 00:00:00+05:30,183.020459,669.488319,267.109799,86.333321,93.801369,391.566108,1691.319375
2023-12-27 00:00:00+05:30,183.281084,675.588128,268.764979,87.995969,92.825873,390.133962,1698.589995
2023-12-28 00:00:00+05:30,186.087839,682.193004,274.297461,89.217242,92.882152,395.009904,1719.687602
2023-12-29 00:00:00+05:30,185.285909,690.002273,277.749969,93.142462,92.814616,399.231484,1738.226712
2024-01-01 00:00:00+05:30,187.671643,687.619350,28.602698,92.568243,93.234827,397.836730,1487.533492
2024-01-02 00:00:00+05:30,188.593870,677.012709,28.446739,94.080910,92.758337,395.522182,1476.414749
2024-01-03 00:00:00+05:30,191.360526,675.134853,27.832364,92.983903,91.981695,392.900976,1472.194315
2024-01-04 00:00:00+05:30,191.019703,671.508646,28.335191,96.463469,94.547994,400.940272,1482.815276
2024-01-05 00:00:00+05:30,190.037346,678.631551,27.862457,96.240632,95.984975,394.288244,1483.045206


In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = df_sectors[[stock_symbol]]
    stock_df.columns = ['Close']

    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
    stock_df['Sector'] = stock_symbol
    stock_df = stock_df.tail(1)
    return stock_df

In [8]:
df_final =  pd.DataFrame()
for sector in sectors:

  df_tmp = stock_prec_dev(sector)
  df_final = df_final.append(df_tmp)
df_final

,Close,200_SMA,Dev%_200,Stock,Max,Min,RSI_14,ATH%,Sector
Date,,,,,,,,,
2024-02-02 00:00:00+05:30,4521.82,3731.0,21.20,AUTO,4521.82,2858.29,75.0,0.00,AUTO
2024-02-02 00:00:00+05:30,1217.01,1231.0,-1.14,BANKS,1313.17,1113.12,40.0,7.90,BANKS
2024-02-02 00:00:00+05:30,2629.80,2651.0,-0.80,FINANCE,2978.63,1989.53,40.0,13.26,FINANCE
2024-02-02 00:00:00+05:30,2422.80,2086.0,16.15,IT,2511.37,1777.84,61.0,3.66,IT
2024-02-02 00:00:00+05:30,1652.20,3469.0,-52.37,FMCG,4237.98,1627.96,18.0,156.51,FMCG


In [12]:
df_sectors['FMCG']

Date
2019-02-04 00:00:00+05:30    1880.260275
2019-02-05 00:00:00+05:30    1882.931462
2019-02-06 00:00:00+05:30    1888.708475
2019-02-07 00:00:00+05:30    1903.472430
2019-02-08 00:00:00+05:30    1843.565345
                                ...     
2024-01-29 00:00:00+05:30    1651.789964
2024-01-30 00:00:00+05:30    1643.781398
2024-01-31 00:00:00+05:30    1662.320838
2024-02-01 00:00:00+05:30    1654.393162
2024-02-02 00:00:00+05:30    1652.201542
Name: FMCG, Length: 1235, dtype: float64